# Track artifacts, in-memory objects & folders [S3 storage]

In [ ]:
import lamindb as ln
import pytest

In [ ]:
ln.setup.login("testuser1")

In [ ]:
try:
    root_path = ln.UPath("s3://lamindb-ci/test-upload")
    if root_path.exists():
        root_path.rmdir()
    ln.setup.delete("testuser1/test-upload", force=True)
except BaseException:  # noqa: S110
    pass

In [ ]:
ln.setup.init(storage="s3://lamindb-ci/test-upload")

## Local artifacts

Some test data.

In [ ]:
pbmc68k = ln.core.datasets.anndata_pbmc68k_reduced()

Subset to a mini artifact to speed up the run time of this notebook:

In [ ]:
pbmc68k = pbmc68k[:5, :5].copy()

In [ ]:
pbmc68k

###  Upload from memory using explicit semantic `key`

#### Upload h5ad

In [ ]:
pbmc68k_h5ad = ln.Artifact.from_anndata(pbmc68k, key="test-upload/pbmc68k.h5ad")

In [ ]:
pbmc68k_h5ad.save()

In [ ]:
pbmc68k_h5ad.delete(permanent=True)

#### Upload zarr

In [ ]:
# Runs too long, should be tested elsewhere
# pbmc68k_zarr = ln.Artifact(pbmc68k, key="test-upload/pbmc68k.zarr", format="zarr")
# ln.save(pbmc68k_zarr)
# pbmc68k_zarr.delete(permanent=True, storage=True)

### Upload using `id` with implicit `key`

#### Upload h5ad

In [ ]:
pbmc68k_h5ad = ln.Artifact.from_anndata(pbmc68k, description="pbmc68k.h5ad")

In [ ]:
pbmc68k_h5ad.save()

In [ ]:
pbmc68k_h5ad.delete(permanent=True, storage=True)

#### Upload zarr

In [ ]:
# Runs too long, should be tested elsewhere
# pbmc68k_zarr = ln.Artifact(pbmc68k, name="pbmc68k.zarr", format="zarr")
# ln.save(pbmc68k_zarr)
# pbmc68k_zarr.delete(permanent=True, storage=True)

### Error behaviors

Specified file does not exist.

In [ ]:
with pytest.raises(FileNotFoundError):
    non_existent_h5ad = ln.Artifact(
        "s3://lamindb-ci/test-upload/non_existent_file.h5ad"
    )

Specified buket does not exist. Normally non-existent bucket raises `FileNotFoundError`, but sometimes strarts to raise `PermissionError`.

In [ ]:
with pytest.raises((FileNotFoundError, PermissionError)):
    non_existent_h5ad = ln.Artifact(
        "s3://non_existent_bucket_6612366/non_existent_file.h5ad"
    )

## Test existing zarr

See `test_artifact.py` for other artifact types.

This should probably go elsewhere:

In [ ]:
# temporarily comment out because of head bucket permission error when
# attempting to get region
# artifact = ln.Artifact("s3://lamindb-ci/lndb-storage/pbmc68k.zarr")
# artifact.save()
# artifact.open()

In [ ]:
ln.setup.delete("test-upload", force=True)